# Lotka-Volterra examples

This notebook shows how to solve the following system:

\begin{equation}
\begin{aligned}
u_{t} &=D_{u} u_{x x}+f(u, v) \\
v_{t} &=D_{v} v_{x x}+g(u, v)
\end{aligned}
\end{equation}

in which $u$ is the prey and $v$ the predator, $D_u$ and $D_v$ are the diffusive coefficients, and $f(u, v)$ and $g(u, v)$ are the reactive terms that can have the following form:

\begin{equation}
\begin{array}{c}
f(u, v)=r u-a u v \\
g(u, v)=b a u v-m v
\end{array}
\end{equation}

with the constants $r, a, b, m > 0$.

1D and 2D cases are shown below.

## 1D case

In [29]:
from pde import (PDE, FieldCollection, PlotTracker, ScalarField, UnitGrid, 
                 CartesianGrid, MemoryStorage)
from pde import ExplicitSolver, ImplicitSolver, Controller

# Species diffusivity coefficients (please be careful with this values)
Du = 1e-6
Dv = 1e-4  # predator assumed to be faster than prey

# Rate coefficients
r = 0.15
a = 0.2
b = 1.02  # Let's predator population increases when they prey because life is not easy
m = 0.05

# Functional response
f_function = f"+ {r} * u - {a} * u * v"  # don't forget to put + (or -) sign in the beginning
g_function = f"+ {b} * {a} * u * v - {m} * v"

# (Dirichlet) Boundary condition example
bc_left = {"value": 0.0}  # both unknowns are set to zero, unfortunately
bc_right = {"value": 0.0}
bc = [bc_left, bc_right]

# Definition of PDE system
eq = PDE(
    {
        "u": f"{Du} * laplace(u)" + f_function,
        "v": f"{Dv} * laplace(v)" + g_function,
    },
    bc=bc  # comment here if you want to "free" the boundaries
)

# Defining the mesh
num_points_in_x = 100
grid = CartesianGrid(bounds=[[0, 1]], shape=num_points_in_x)

# Initialize state (Initial Conditions)
u = ScalarField.from_expression(grid, "sin(pi * x)", label="Prey")
# v = ScalarField.from_expression(grid, "abs(sin(2 * pi * x))", label="Predator")
v = ScalarField.from_expression(grid, "0.1", label="Predator")
# v = ScalarField(grid, 0.1, label="Predator")
state = FieldCollection([u, v])  # state vector

# Define time tracker to plot and animate
x_axis_limits = (0, 1)
y_axis_limits = (0, 3.5)
tracker_plot_config = PlotTracker(show=True, plot_args={
        'ax_style': {'xlim': x_axis_limits, 'ylim': y_axis_limits},
    }
)
storage = MemoryStorage()
trackers = [
    "progress",  # show progress bar during simulation
    "steady_state",  # abort if steady state is reached
    storage.tracker(interval=1),  # store data every simulation time unit
    tracker_plot_config,  # show images during simulation
]

# Setup explicit solver
explicit_solver = ExplicitSolver(eq)
controller = Controller(explicit_solver, t_range=[0, 100], tracker=trackers)
solve = controller.run(state, dt=1e-2)

Output()

Spent more time on handling trackers (72.25120208600015) than on the actual simulation (2.9689108959998975)


In [32]:
u_storage = storage.extract_field(0)
v_storage = storage.extract_field(1)

u_storage.data

[array([0.01570732, 0.04710645, 0.0784591 , 0.10973431, 0.14090123,
        0.1719291 , 0.2027873 , 0.23344536, 0.26387305, 0.29404033,
        0.32391742, 0.35347484, 0.38268343, 0.41151436, 0.43993917,
        0.46792981, 0.49545867, 0.52249856, 0.54902282, 0.57500525,
        0.60042023, 0.62524266, 0.64944805, 0.67301251, 0.6959128 ,
        0.7181263 , 0.73963109, 0.76040597, 0.78043041, 0.79968466,
        0.81814972, 0.83580736, 0.85264016, 0.86863151, 0.88376563,
        0.89802758, 0.91140328, 0.92387953, 0.93544403, 0.94608536,
        0.95579301, 0.96455742, 0.97236992, 0.97922281, 0.98510933,
        0.99002366, 0.99396096, 0.99691733, 0.99888987, 0.99987663,
        0.99987663, 0.99888987, 0.99691733, 0.99396096, 0.99002366,
        0.98510933, 0.97922281, 0.97236992, 0.96455742, 0.95579301,
        0.94608536, 0.93544403, 0.92387953, 0.91140328, 0.89802758,
        0.88376563, 0.86863151, 0.85264016, 0.83580736, 0.81814972,
        0.79968466, 0.78043041, 0.76040597, 0.73

In [36]:
from pde.visualization import movie

movie(v_storage, "v_field.mp4")

## 2D case

In this case, you just have to change the definition of your grid. Everything else remains the same.

In [2]:
from pde import (PDE, FieldCollection, PlotTracker, ScalarField, UnitGrid, 
                 CartesianGrid, MemoryStorage)
from pde import ExplicitSolver, ImplicitSolver, Controller

# Species diffusivity coefficients (please be careful with these values)
Du = 1e-6
Dv = 1e-4  # predator assumed to be faster than prey

# Rate coefficients
r = 0.15
a = 0.2
b = 1.02  # Let's predator population increases when they prey because life is not easy
m = 0.05

# Functional response
f_function = f"+ {r} * u - {a} * u * v"  # don't forget to put + (or -) sign in the beginning
g_function = f"+ {b} * {a} * u * v - {m} * v"

# (Dirichlet) Boundary condition example
bc_x = ({'value': 0}, {'value': 0})  # left right
bc_y = ({'value': 0}, {'value': 0})  # bottom top
bc = [bc_x, bc_y]

# Definition of PDE system
eq = PDE(
    {
        "u": f"{Du} * laplace(u)" + f_function,
        "v": f"{Dv} * laplace(v)" + g_function,
    },
    bc=bc
)

#################################################################
# Defining the mesh ** (HERE IS DIFFERENT COMPARED TO 1D CASE) **
num_points_in_x = 100
num_points_in_y = 100
mesh_shape = (num_points_in_x, num_points_in_y)
grid = CartesianGrid(bounds=[[0, 1], [0, 1]], shape=mesh_shape)
#################################################################

# Initialize state (Initial Conditions)
# u = ScalarField(grid, u_0, label="Prey")  # with this one you can give a value for all the domain
u = ScalarField.from_expression(grid, "sin(pi * x) * sin(pi * y)", label="Prey")
# v = ScalarField.from_expression(grid, "abs(sin(2 * pi * x) * sin(2 * pi * y))", label="Predator")
v = ScalarField(grid, 0.1, label="Predator")
state = FieldCollection([u, v])  # state vector

# Define time tracker to plot and animate
tracker_plot_config = PlotTracker(plot_args={
        "vmin": 0,  # min value in the color bar
        "vmax": 3   # max value in the color bar
    }
)
storage = MemoryStorage()
trackers = [
    "progress",  # show progress bar during simulation
    "steady_state",  # abort if steady state is reached
    storage.tracker(interval=1),  # store data every simulation time unit
    tracker_plot_config,  # show images during simulation
]

# Setup explicit solver
explicit_solver = ExplicitSolver(eq)
controller = Controller(explicit_solver, t_range=[0, 300], tracker=trackers)
solve = controller.run(state, dt=1e-2)

# Setup implicit solver (if explicit does not work, try this one)
# implicit_solver = ImplicitSolver(eq)
# controller = Controller(implicit_solver, t_range=[0, 50], tracker=trackers)
# solve = controller.run(state)

# print(controller.diagnostics)  # to debug

Output()

Spent more time on handling trackers (261.0201935850001) than on the actual simulation (9.020186543999927)


In [28]:
u_storage = storage.extract_field(0)
v_storage = storage.extract_field(1)

grid.axes_coords

(array([0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085,
        0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175,
        0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265,
        0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355,
        0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445,
        0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535,
        0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625,
        0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715,
        0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805,
        0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895,
        0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985,
        0.995]),
 array([0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085,
        0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175,
        0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.2